In [3]:
import string
import numpy as np
import tensorflow as tf
from tensorflow.keras import initializers
from keras.models import Model
from tensorflow.keras.utils import Sequence
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import scipy.sparse

In [4]:
books = [1]
window_size = 2

In [5]:
book_text = ""

for book in books:
  with open (f"harry_potter/HP{book}.txt", 'r') as f:
      data = str(f.read()).lower()
      book_text += data + ' '
      f.close()

formatted = re.compile(r'[A-Za-z]+[\w^\']*|[\w^\']*[A-Za-z]+[\w^\']*')
formatted = formatted.findall(book_text.lower())
book_text = None

In [6]:
unique_words = np.unique(formatted)
dict_size = len(unique_words)
corpus_size = len(formatted)
print(f"Vocab size : {dict_size}\nTotal length of corpus : {corpus_size}")

Vocab size : 5729
Total length of corpus : 80591


In [7]:
encoding_to_word = {}
word_to_encoding = {}

for i, word in enumerate(unique_words):
    encoding_to_word[i] = word
    word_to_encoding[word] = i

In [8]:
window = [i for i in range(-window_size, window_size + 1) if i != 0]
data = []
labels = []

for i in range(corpus_size):
    for j in window:
        if i + j >= 0 and i + j < corpus_size:
            data_enc = np.zeros(dict_size)
            labels_enc = np.zeros(dict_size)
            data_enc[word_to_encoding[formatted[i]]] = 1
            labels_enc[word_to_encoding[formatted[i + j]]] = 1
            data.append(data_enc)
            labels.append(labels_enc)
data = np.array(data)
labels = np.array(labels)

In [ ]:
embedding_size = 300
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=embedding_size,
                          activation='linear',
                          input_shape=(dict_size,),
                          name='hidden_1'),
    # Modify the Reshape layer to reshape the output to (None, embedding_size)
    tf.keras.layers.Reshape((embedding_size,), name='reshape'),
    tf.keras.layers.Dense(units=dict_size,
                          activation='softmax',
                          name='output')
])

In [ ]:
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
loss_function = tf.keras.losses.CategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

In [ ]:
epochs = 10
model.fit(datagen, epochs=epochs)

In [ ]:
def get_embedding(model, input_word):
    hidden_layer_model = Model(inputs=model.input, outputs=model.layers[0].output)
    hidden_output = hidden_layer_model.predict(np.array([encoding_mapping[input_word]]), verbose = 0)
    return hidden_output[0]

In [ ]:
embedding_collection = {}

for word in unique_words:
  embedding_collection[word] = get_embedding(model, word)

In [ ]:
#Find the word most similar to "water"
min_dist = np.linalg.norm(embedding_collection["water"] - embedding_collection["after"])
min_word = "after"
for word in unique_words:
  if word != "water" and word != "after":
    min_dist = min(min_dist, np.linalg.norm(embedding_collection["water"] - embedding_collection[word]))
    min_word = word

min_word

In [ ]:
# #Get the most similar word to every other word
# for i in range(len(unique_words)):
#   min_dist = np.infty
#   min_word = ""
#   for j in range(len(unique_words)):
#     if j != i:
#       new_min = np.linalg.norm(embedding_collection[unique_words[i]] - embedding_collection[unique_words[j]])

#       if new_min <= min_dist:
#         min_dist = new_min
#         min_word = unique_words[j]

#   print(unique_words[i] + ' -> ' + min_word)